# Deformation map

This notebook will show how to compute a deformation map, i.e. $F_\text{calc}^\text{TAAM} - F_\text{calc}^\text{IAM}$

## Setup: structure definition

We first load the structure information from a .cif file or a .pdb file. Here, an ethanol molecule in a P1 box is provided, but this can be replaced with whatever structure you have

In [ ]:
from pathlib import Path

# Replace this with a path to your structure file
structure_file = Path().resolve() / "ethanol.pdb"

In [ ]:
# This cell generates the example pdb file, if it does not exist
if not structure_file.exists():
    structure_file.write_text("""
CRYST1   10.000   10.000   10.000  90.00  90.00  90.00 P 1
SCALE1      0.100000  0.000000  0.000000        0.00000
SCALE2      0.000000  0.100000  0.000000        0.00000
SCALE3      0.000000  0.000000  0.100000        0.00000
ATOM      1 C1    C1     1      -0.756   0.000   0.000  1.00  0.00           C
ATOM      2 C2    C2     2       0.756   0.000   0.000  1.00  0.00           C
ATOM      3 H1    H1     3      -1.140   0.659   0.785  1.00  0.00           H
ATOM      4 H2    H2     4      -1.140   0.350  -0.963  1.00  0.00           H
ATOM      5 H3    H3     5      -1.141  -1.009   0.178  1.00  0.00           H
ATOM      6 H4    H4     6       1.140  -0.350   0.963  1.00  0.00           H
ATOM      7 H5    H5     7       1.141   1.009  -0.178  1.00  0.00           H
ATOM      8 H6    H6     8       1.140  -0.659  -0.785  1.00  0.00           H
END
""")

## Compute deformation map

Next, we initialize an IAM calculator and a TAAM calculator, set the resolution we want, and compute a deformation map. This is then saved in an .mtz-file.

In [ ]:
import pydiscamb
from cctbx import miller
from iotbx import pdb

# Load structure
xrs = pdb.input(file_name=str(structure_file)).xray_structure_simple()
xrs.scattering_type_registry(table="xray")

# Prepare calculators
iam_calculator = pydiscamb.DiscambWrapper(
    xrs,
    method=pydiscamb.FCalcMethod.IAM,
)
taam_calculator = pydiscamb.DiscambWrapper(
    xrs,
    method=pydiscamb.FCalcMethod.TAAM,
)

# Set resolution
miller_set = miller.build_set(
    crystal_symmetry=xrs.crystal_symmetry(), 
    anomalous_flag=False, 
    d_min=0.3,
)
iam_calculator.set_indices(miller_set.indices())
taam_calculator.set_indices(miller_set.indices())

# Compute structure factors
fcalc_iam = iam_calculator.f_calc()
fcalc_taam = taam_calculator.f_calc()

# Compute deformation map
deformation_map = fcalc_taam - fcalc_iam

# Create a miller.array to save
deformation_map_array = miller_set.array(
    data=deformation_map,
)
deformation_map_array.write_mtz(
    file_name="deformation_map.mtz",
)